In [1]:
from llama_cpp import Llama

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("../models/mistral_finetuned")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
model.model.embed_tokens.weight.data.shape

torch.Size([32128, 4096])

In [29]:
tokenizer = AutoTokenizer.from_pretrained("../models/mistral_finetuned_completion_only")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
new_tokens = [f"<pad{i}>" for i in range(32128-32002)]

In [31]:
tokenizer.add_tokens(new_tokens)

tokenizer.save_pretrained('../models/mistral_finetuned_completion_only')
# tokenizer.save_vocabulary('../models/mistral_finetuned')

('../models/mistral_finetuned_completion_only/tokenizer_config.json',
 '../models/mistral_finetuned_completion_only/special_tokens_map.json',
 '../models/mistral_finetuned_completion_only/tokenizer.json')

In [28]:
tokenizer.vocab_size

32000

In [32]:
model = Llama(model_path='../models/mistral_finetuned/mistral_finetuned.gguf',
              n_ctx=3000,            # context window size
              use_mlock=False)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/mistral_finetuned/mistral_finetuned.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention

In [3]:
prompt = '<s> [INST] Tell me a recipe for a mai tai. [/INST]'
output = model(prompt=prompt, max_tokens=250, temperature=0.7, top_k=50, top_p=0.95)
output

KeyboardInterrupt: 

In [7]:

print(output['choices'][0]['text'])

 Sure, I'd be happy to share a recipe for a classic Mai Tai cocktail with you! Here's what you'll need:

Ingredients:

* 1.5 oz white rum
* 0.75 oz orange curaçao
* 0.5 oz lime juice (freshly squeezed)
* 0.25 oz orgeat syrup
* 0.25 oz simple syrup
* Garnish: mint sprig, lime wheel, and/or a cherry

Instructions:

1. Fill a cocktail shaker with ice.
2. Add the white rum, orange curaçao, lime juice, orgeat syrup, and simple syrup to the shaker.
3. Shake well for about 15-20 seconds until all the ingredients are combined.
4. Strain the mixture into a rocks glass filled with ice.
5. Garnish with a mint sprig, lime wheel, and/or a cherry.
6. Serve immediately and enjoy your classic Mai Tai!

Note: The original


In [60]:
import evaluate
metric = evaluate.load('rouge')
predictions = ["Dima I asd as  asd as das das das das das askadsl dklask lkdasj dasjk dklasjd"]
reference = ["Dima I"]
results = metric.compute(predictions=predictions, references=reference)

In [61]:
results

{'rouge1': 0.2222222222222222,
 'rouge2': 0.125,
 'rougeL': 0.2222222222222222,
 'rougeLsum': 0.2222222222222222}

In [62]:
from rouge_score import rouge_scorer

In [65]:
score = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True).score(reference[0], predictions[0])

In [67]:
score['rouge1'].recall

1.0